The following notebook contains the code  for the NLP part.
 Model used is as follows:
 1. embedding layer
 2. linear layer
 3. lstm
 4. linear layer

hyperparamaters for simple model:

batch size=256

learning rate=0.001

num epoch=10



hyperparamaters for probe model:

batch size= 100

learning rate=0.001

num epoch=

probe layer and its accuracy:
1. word embedding- 65%
2. linear layer-57%
3. lstm-59%

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from torchtext.legacy import data
from torchtext.legacy import datasets
import pandas as pd
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
device='cuda'
import pandas as pd

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
text = data.Field(tokenize = 'spacy', lower = True)
label = data.LabelField()
train_data, val_data,test_data= datasets.SNLI.splits(text,label)

downloading snli_1.0.zip


snli_1.0.zip: 100%|██████████| 94.6M/94.6M [00:01<00:00, 75.8MB/s]


extracting


In [3]:
text.build_vocab(train_data, val_data)
label.build_vocab(train_data)
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data), batch_size=256)

In [10]:
len(train_iter)

2146

In [95]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, 300, padding_idx = 1)
        self.fc1 = nn.Linear(300, 300)
        self.lstm = nn.LSTM(300, 300, num_layers = 1, bidirectional = True, dropout=0.5 )
     
        self.fc2 = nn.Linear(1200 , 3)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, prem, hypo):

        p = self.embedding(prem)
        h = self.embedding(hypo)
        
        p = F.relu(self.fc1(p))
        h = F.relu(self.fc1(h))
        
        o_p, (h_p, c_p) = self.lstm(p)
        o_h, (h_h, c_h) = self.lstm(h)

        h_p = torch.cat((h_p[-1], h_p[-2]), dim=-1)
        h_h = torch.cat((h_h[-1], h_h[-2]), dim=-1)
        
        hidden = torch.cat((h_p, h_h), dim=1)

        prediction = self.fc2(hidden)
        return prediction

input_dim = len(text.vocab)



In [109]:
num_epochs=3
model = Model(input_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
model

Model(
  (embedding): Embedding(15, 300, padding_idx=1)
  (fc1): Linear(in_features=300, out_features=300, bias=True)
  (lstm): LSTM(300, 300, num_layers=2, dropout=0.25, bidirectional=True)
  (fc2): Linear(in_features=1200, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [97]:
def eval( iterator):
    
    loss = 0
    acc = 0
    correct=0
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            prem = batch.premise.to(device)
            hypo = batch.hypothesis.to(device)
            labels = batch.label.to(device)
                        
            predictions = model(prem, hypo)
            
            loss += criterion(predictions, labels).item()
            correct += (predictions.argmax(1) == labels).type(torch.float).sum().item()
            
        acc= correct/len(val_data)*100
        
    return  acc 

In [110]:

n_total_steps = len(train_iter)
for epoch in range(num_epochs):
    model.train()
    ep_loss=0
    acc=0
    correct=0
    total=0
    for batch in train_iter:
        prem = batch.premise.to(device)
        hypo = batch.hypothesis.to(device)
        labels = batch.label.to(device)
                        
        outputs = model(prem, hypo)
            
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()        
        
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        ep_loss += loss.item()
     
    accu= 100 * correct / len(train_data)
    val_accu=eval(val_iter)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], train_accuracy: {accu:.4f} ,train_loss: {ep_loss/len(train_iter):.4f}, val_accuracy: {val_accu:.4f} ')  

Epoch [1/3], train_accuracy: 63.7199 ,train_loss: 0.8098, val_accuracy: 66.7547 
Epoch [2/3], train_accuracy: 68.1918 ,train_loss: 0.7339, val_accuracy: 68.3601 
Epoch [3/3], train_accuracy: 70.0515 ,train_loss: 0.6995, val_accuracy: 69.3863 


In [132]:
test_accu= eval(test_iter)

In [133]:
print(test_accu)

68.0857549278602


In [134]:
PATH = './nlp.pth'
torch.save(model.state_dict(), PATH)

In [135]:
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [114]:
def tagger(data,x):
  psent=[]
  ptags=[]
  hsent=[]
  htags=[]

  for i, batch in enumerate(data[0:x]):
 
    ps, pt = zip(*nltk.pos_tag(batch.premise))
    psent.append(ps)
    ptags.append(pt)

    hs, ht = zip(*nltk.pos_tag(batch.hypothesis))
    hsent.append(hs)
    htags.append(ht)
  return psent,ptags,hsent,htags
psent,ptags,hsent,htags= tagger(train_data, 20000)
psent_test,ptags_test,hsent_test,htags_test= tagger(test_data, 9000)

In [115]:
import collections
from torchtext.data.functional import numericalize_tokens_from_iterator
from torchtext.vocab import build_vocab_from_iterator
vocab = build_vocab_from_iterator(psent+hsent+psent_test+hsent_test, specials=["<unk>"])
tag_vocab = build_vocab_from_iterator(ptags+htags+ptags_test+htags_test, specials=["<unk>"])

def convert_to_ids(sentences, taggings):
  
  for sentence, tagging in zip(sentences, taggings):
    
    sentence_iter = (numericalize_tokens_from_iterator(vocab,sentences))
    tagging_iter = (numericalize_tokens_from_iterator(tag_vocab,taggings))
    
  return list(sentence_iter), list(tagging_iter)

ptrain_sent, ptrain_tag = convert_to_ids(psent, ptags)
htrain_sent, htrain_tag = convert_to_ids(hsent, htags)
ptest_sent, ptest_tag = convert_to_ids(psent_test, ptags_test)
htest_sent, htest_tag = convert_to_ids(hsent_test, htags_test)



In [12]:
len(tag_vocab)

45

In [116]:
def make_list(input):
  x=[]
  for i in input:
    x.append([n for n in i])
  return x

def make_pairs(x,y):
  da=[]
  for a,b in zip(x,y):

    l=[a,b]
    l=torch.tensor(l)
    da.append(l)  
  return da

pts=make_list(ptrain_sent)
ptt=make_list(ptrain_tag)
hts= make_list(htrain_sent)
htt=  make_list(htrain_tag)

pts_test=make_list(ptest_sent)
ptt_test=make_list(ptest_tag)
hts_test= make_list(htest_sent)
htt_test=  make_list(htest_tag)

prem= make_pairs(pts,ptt)
hypo= make_pairs(hts,htt)
prem_test= make_pairs(pts_test,ptt_test)
hypo_test= make_pairs(hts_test,htt_test)

In [156]:
def collate_fn(items):
  max_len = max(len(item[0]) for item in items)

  sentences = torch.zeros((len(items), max_len)).long()
  taggings = torch.zeros((len(items), max_len)).long()

  for i, (sentence, tagging) in enumerate(items):
    sentences[i][0:len(sentence)] = sentence
    taggings[i][0:len(tagging)] = tagging

  return sentences, taggings

  
ptraindataloader=DataLoader(prem,batch_size=100, collate_fn=collate_fn)
htraindataloader=DataLoader(hypo,batch_size=100, collate_fn=collate_fn)
ptestdataloader=DataLoader(prem_test,batch_size=100, collate_fn=collate_fn)
htestdataloader=DataLoader(hypo_test,batch_size=100, collate_fn=collate_fn)

In [207]:
class probe(nn.Module):
    def __init__(self, model,out):
        super(probe, self).__init__()
        self.features = nn.Sequential(*list(model.children())[:-2])
        for p in model.parameters():
          p.requires_grad=False
        self.pr= nn.Linear(600,out)
    def forward(self, prem, hypo):
        p,(_,_) = self.features(prem)
        h,(_,_)= self.features(hypo)
        p= self.pr(p)
        h= self.pr(h)
        
        return p,h


probe_model = probe(model,len(tag_vocab)).to(device)


In [208]:
print(probe_model)
criterion1 = nn.CrossEntropyLoss().to(device)
optimizer1 = torch.optim.Adam(probe_model.parameters(), lr=0.001,weight_decay=1e-4)

probe(
  (features): Sequential(
    (0): Embedding(33932, 300, padding_idx=1)
    (1): Linear(in_features=300, out_features=300, bias=True)
    (2): LSTM(300, 300, dropout=0.5, bidirectional=True)
  )
  (pr): Linear(in_features=600, out_features=46, bias=True)
)


In [209]:
#create a test function
def evaluate_probe(model):
    epoch_loss = 0
    epoch_acc = 0
    correct=0
    total=0
    
    l1=0
    l2=0
    t1=0
    t2=0
    model.eval()
    
    with torch.no_grad():
    
        for i, (p,h) in enumerate(zip(ptestdataloader,htestdataloader)) :

            ps=p[0].to(device)
            pt=p[1].to(device)
            hs=h[0].to(device)
            ht=h[1].to(device)
                
            tag1, tag2 = model(ps,hs)
         
          
            x=[]
            for i in range(100):
              _, predicted = tag1[i].max(1)
              x.append(predicted)
           
            x=torch.stack(x)
           
            
            y=[]
            for i in range(100):
              _, pred = tag2[i].max(1)
              y.append(pred)
      
            y=torch.stack(y)
            
            
            l1+= ((pt==x)|(pt==0)).sum()
            l2+= ((ht==y)|(ht==0)).sum()
            t1+= ((pt!=-1)|(pt==0)).sum()
            t2+= ((ht!=-1)|(ht==0)).sum()
            
            
        correct=l1+l2
        total=t1+t2
        epoch_acc=(correct/total)*100
        
    print(f' test_accuracy: {epoch_acc:.4f}')  

In [210]:
def train_probe(model, epoch, num_epochs):
    epoch_loss = 0
    epoch_acc = 0
    correct=0
    total=0
    pc=0
    hc=0
    ptest_loss=0
    htest_loss=0
    l1=0
    l2=0
    t1=0
    t2=0
    
    model.train()
    
    
    
    for i, (p,h) in enumerate(zip(ptraindataloader,htraindataloader)) :

       ps=p[0].to(device)
       pt=p[1].to(device)
       hs=h[0].to(device)
       ht=h[1].to(device)
                    
       tag1, tag2 = model(ps,hs)
       
        
       x=[]
       for i in range(100):
        _, predicted = tag1[i].max(1)
        x.append(predicted)
           
       x=torch.stack(x)
       
        
           
       pc +=x.eq(pt).sum().item()
       y=[]
       for i in range(100):
         _, pred = tag2[i].max(1)
         y.append(pred)
      
       y=torch.stack(y)
          
       
       l1+= ((pt==x)|(pt==0)).sum()
       l2+= ((ht==y)|(ht==0)).sum()
       t1+= ((pt!=-1)|(pt==0)).sum()
       t2+= ((ht!=-1)|(ht==0)).sum()
       pt = torch.zeros((len(pt),46)).to(device).long()
       ht = torch.zeros((len(ht),46)).to(device).long() 
       tag1= torch.squeeze(tag1,dim=-1)
       pt= torch.squeeze(pt,dim=-1)
       tag2= torch.squeeze(tag2,dim=-1)
       ht= torch.squeeze(ht,dim=-1)
       #print(tag1)
       ploss=criterion1(tag1,pt)
       hloss=criterion1(tag2,ht)     
       optimizer1.zero_grad()
       loss=(ploss+hloss)
       loss.backward()
       #hloss.backward()
       optimizer1.step()     
       
            
    correct=l1+l2
    total=t1+t2
    epoch_acc=(correct/total)*100
        
    print(f'Epoch [{epoch+1}/{num_epochs1}], train_accuracy: {epoch_acc:.4f} ,train_loss: {(ploss+hloss)/2:.4f} ')  
       

In [ ]:
num_epochs1=10
for epoch in range(num_epochs1):
  train_probe(probe_model,epoch,num_epochs1)
  evaluate_probe(probe_model)